In [2]:
import pandas as pd
from pandas import DataFrame
from data_import import initialize,connection
from functions import get_connection , use_database
from constants import *
from pandasql import sqldf
from sqlalchemy.engine import Engine, Connection
from sqlalchemy.sql import text
from sqlalchemy import select ,MetaData, Table, and_, insert


In [3]:
connection: Engine=get_connection(host_name=HOST_NAME, user_name=USER_NAME, user_password=USER_PASSWORD)
connection=use_database(connection=connection, database_name=GENDER_BASE_VIOLENCE_DATA_BASE_NAME)

In [4]:
calls_df=pd.read_csv(r'/Users/angelaivonnesalgadobeltran/Documents/IronHack/Final project/cleaning_data/final_data_calls.csv')

1. We need an index from the very begining in order to keep track of calls observation. DataFrame [Index] will be the staging table index. We index to start from 1 intead of 0. 

In [42]:
calls_df.index += 1
calls_df.to_sql(name="staging", index=True, index_label='id',
                con=connection, if_exists='replace')

59360

2. Feeding localization table from a new dataframe. extract distinc values and inserting to SQL

In [5]:
location: DataFrame=calls_df[['prov_residencia_persona_en_situacion_violencia', 'media_edad']].drop_duplicates()
location.rename(columns={'prov_residencia_persona_en_situacion_violencia':'region_name', 'media_edad':'mean_age'}, inplace=True)
location.reset_index(drop=True, inplace=True)


location.to_sql(name=LOCALIZATION_TABLE_NAME, index=False,
                con=connection, if_exists='append')

3. Feeding calls table. Unfortunately this method is to heavy and not efficient for this procedure. 
We needed to loop on each row and insert it based on conditions ---> database conection is too long (approximately 10 000 rec per 20min). 
We decided to insert the whole data source into one srtaging table and perform feeding through SQL queries. 

In [ ]:
def insert_into_calls(conn: Connection, row):
    metadata = MetaData(bind=None)
    calls = Table(
        'calls',
        metadata,
        autoload=True,
        autoload_with=connection
    )
    result = conn.execute(insert(calls),
                            {
                            "call_date": {row['fecha']},
                            "id_localization": {row['id_localization']},
                            "victim_gender": {row['genero_persona_en_situacion_de_violencia']},
                            "victim_age": {row['edad']},
                            "aggressor_gender": {row['genero_de_la_persona_agresora']},
                            "link": {row['vinculo_con_la_persona_agresora']}
    })


def get_localization_id(conn:Connection,region_name) -> int:
    metadata = MetaData(bind=None)
    localization = Table(
        'localization',
        metadata,
        autoload=True,
        autoload_with=connection
    )
    statement = localization.select().with_only_columns(localization.columns.id).where(
        localization.columns['region_name'] == region_name)

    return conn.execute(statement=statement).first()[0]

def insert_into_call():
    with connection.begin() as conn:
        for _, row in calls_df.iterrows():
            id_localization = get_localization_id(conn,
                row['prov_residencia_persona_en_situacion_violencia'])
            print(id_localization)
            row['id_localization'] = id_localization
            insert_into_calls(conn=conn,row=row)


# not used at the end --> too slow , feeding through feeding.sql script 